<a href="https://colab.research.google.com/github/ARMargolis/matrix_bin_packing/blob/main/Matrix_Bin_Packing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Matrix Multiplication for Efficient Bin Packing
In this workbook, I develop a new method for efficient bin packing using matrix multiplication. Bin packing is one of many optimization problems with real world applications. There are heuristic algorithms that come within a few percentage points of the optimal answer, but for many business use cases at scale, that small percentage can mean a large loss of value.
I take an approach of using high-memory to efficiently test a large number of combinations to find a better solution than standard heuristic approaches. The first version uses numpy, but future versions will use GPUs and TPUs.


Example problems from

 https://people.sc.fsu.edu/~jburkardt/datasets/bin_packing/bin_packing.html (0 to 3)
https://scipbook.readthedocs.io/en/latest/bpp.html (4)
http://www.or.deis.unibo.it/staff_pages/martello/Slides_Estoril_Martello.pdf (5 and 6)

In [1]:
bin_problems=[(100, [70, 60, 50, 33, 33, 33, 11, 7, 3]),
              (100, [99, 94, 79, 64, 50, 46, 43, 37, 32, 19, 18, 7, 6, 3]),
              (100, [49, 41, 34, 33, 29, 26, 26, 22, 20, 19]),
              (524, [442, 252, 252, 252, 252, 252, 252, 252, 127, 127, 127, 127, 127, 106,
                   106, 106, 106, 85, 84, 46, 37, 37, 12, 12, 12, 10, 10, 10, 10, 10, 10, 9]),
              (9, [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]),
              (100, [99, 94, 90, 88, 80, 10, 10, 6, 5, 5, 4, 4]),
              (100, [49, 41, 34, 33, 29, 26, 26, 22, 20, 19])]

In [2]:
import time, numpy as np
import itertools
from math import factorial

def binpack(bin_cap, articles, noisy=False, limit=0, threshold=.995, max_rows=1e6):
  arts2sort=np.sort(np.array(articles),kind='mergesort')
  bin_list=[]
  cycles=0
  while arts2sort.sum()>bin_cap and (limit>cycles or limit==0):
    cycles+=1
    #Max_items: What is the most items you can fit (by starting with the smallest and counting up)
    #Min_items: Start with the top and count down
    max_items=np.where(np.cumsum(arts2sort)>bin_cap)[-1][0]
    min_items=np.where(np.cumsum(np.flip(arts2sort))>bin_cap)[-1][0]
    if noisy:
      print(arts2sort, '\nMin:', min_items, 'Max:', max_items)
    
    #If you can fit more of some items than others, try different amounts until
    if max_items>min_items:
      max_pack=[np.concatenate([np.zeros(arts2sort.shape[0]-min_items, dtype=bool), np.ones(min_items, dtype=bool)])]
      max_load=[arts2sort[-min_items:].sum()]
      m=min_items+1
      #Starting with min_items, try every amount until you get a size that fits within threshold
      while m<=max_items and max_load[-1]<bin_cap*threshold:
        #Make every combination of m items from arts2sort
        num_combos=int(factorial(arts2sort.shape[0])/(factorial(arts2sort.shape[0]-m)*factorial(m)))
        if num_combos<=max_rows:
          combos=list(itertools.combinations(range(arts2sort.shape[0]), m))
          combos_tup=[tuple(itertools.chain.from_iterable([n]*m for n in range(num_combos))),
                  tuple(itertools.chain.from_iterable(combos))]

          #Make a matrix of m ones and the rest zeros, with the ones being every combo
          bin_matrix=np.zeros((num_combos, arts2sort.shape[0]), dtype=bool)
          bin_matrix[combos_tup]=1

        else:
          avoid=0
          #If the number of combinations would use too much memory, include all of the 
          #largest items that fit in a bin (min_items), ignore the next largest item,
          #And ignore the first few items (avoid) until the number of combinations is
          #less thanbmax_rows.
          while num_combos>max_rows:
            avoid+=1
            num_combos=int(factorial(arts2sort.shape[0]-avoid-min_items)/(factorial(arts2sort.shape[0]-avoid-m)*factorial(m-min_items)))
          if noisy:
            print('avoid:', avoid, 'try:', arts2sort.shape[0]-avoid-min_items, 'num_combos:', num_combos)
          combos=list(itertools.combinations(range(arts2sort.shape[0]-avoid-min_items), m-min_items))
          combos_tup=[tuple(itertools.chain.from_iterable([n]*(m-min_items) for n in range(num_combos))),
                  tuple(itertools.chain.from_iterable(combos))]

          #Make a matrix of m-min_items-avoid ones and the rest zeros, with the ones being
          # every combo that we are going to test, then put a matrix of zeroes in front
          #(those are the first items we're going to ignore) and a matrix of ones in the
          #back, because we're including the min_items largest items
          bin_matrix=np.zeros((num_combos, arts2sort.shape[0]-min_items-avoid+1), dtype=bool)
          if noisy:
            print(bin_matrix.shape, len(combos_tup[0]), len(combos_tup[1]))
          bin_matrix[combos_tup]=1
          bin_matrix=np.concatenate([np.zeros((num_combos, avoid-1), dtype=bool),
                          bin_matrix, np.ones((num_combos,min_items), dtype=bool)], axis=1)
          if noisy:
            print('bin_matrix.shape:', bin_matrix.shape)

        #Find out how much load they carry, then sort them and choose the highest load that fits in a bin
        pack_opts=np.matmul(bin_matrix, arts2sort)
        pack_order=pack_opts.argsort(kind='mergesort')
        best_opt=np.where(pack_opts[pack_order]<=bin_cap)[-1][-1]
        max_pack.append(bin_matrix[pack_order[best_opt]])
        max_load.append(pack_opts[pack_order[best_opt]])
        m+=1
        min_items=np.where(np.cumsum(np.flip(arts2sort))>bin_cap)[-1][0]
        if noisy:
          print("best_opt:",best_opt,"load:",pack_opts[pack_order[best_opt]],
                "used:", arts2sort[bin_matrix[pack_order[best_opt]]])         
      if noisy:
        print('max_load:', max_load, max(max_load))
      #After trying different numbers of items, choose the best one and the best combination with that many items
      best_m=max_load.index(max(max_load))
      arts2add=np.arange(arts2sort.shape[0])[max_pack[best_m]]
      if noisy:
        print('best_m:', best_m, 'best_load:', max_load[best_m], max_pack[best_m], '\narts2add:', arts2add)

      #Add those items to the bin_list and remove from arts2sort
      bin_list.append(arts2sort[arts2add])
      arts2sort=np.delete(arts2sort,arts2add)

    #If you can only fit one amount of items, just choose the largest items
    else:
      bin_list.append(arts2sort[-max_items:])
      arts2sort=np.delete(arts2sort,np.arange(arts2sort.shape[0]-max_items,arts2sort.shape[0]))

  return bin_list+[arts2sort]

Now let's test it for performance!

In [3]:
for (cap, articles) in bin_problems:
  print(time.ctime(), '\nCap:', cap, 'Number of articles:', len(articles),
        'Total load:', sum(articles))
  result=binpack(cap, articles, noisy=False)
  print('Resulting length:', len(result),'\n', [list(res) for res in result], '\n')
print('Done:', time.ctime())

Tue Mar 30 13:47:38 2021 
Cap: 100 Number of articles: 9 Total load: 300
Resulting length: 4 
 [[7, 33, 60], [3, 11, 33, 50], [70], [33]] 

Tue Mar 30 13:47:38 2021 
Cap: 100 Number of articles: 14 Total load: 597
Resulting length: 7 
 [[6, 94], [18, 32, 50], [99], [19, 37, 43], [3, 7, 79], [64], [46]] 

Tue Mar 30 13:47:38 2021 
Cap: 100 Number of articles: 10 Total load: 299
Resulting length: 3 
 [[26, 33, 41], [22, 29, 49], [19, 20, 26, 34]] 

Tue Mar 30 13:47:38 2021 
Cap: 524 Number of articles: 32 Total load: 3659


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Resulting length: 7 
 [[10, 10, 252, 252], [10, 10, 252, 252], [10, 10, 252, 252], [37, 106, 127, 252], [37, 106, 127, 127, 127], [12, 12, 12, 46, 442], [9, 84, 85, 106, 106, 127]] 

Tue Mar 30 13:47:38 2021 
Cap: 9 Number of articles: 24 Total load: 110
Resulting length: 13 
 [[4, 5], [4, 5], [4, 5], [4, 5], [4, 5], [4, 5], [3, 6], [3, 6], [2, 7], [2, 7], [8], [8], [2, 2]] 

Tue Mar 30 13:47:38 2021 
Cap: 100 Number of articles: 12 Total load: 495
Resulting length: 5 
 [[10, 90], [6, 94], [5, 5, 10, 80], [99], [4, 4, 88]] 

Tue Mar 30 13:47:38 2021 
Cap: 100 Number of articles: 10 Total load: 299
Resulting length: 3 
 [[26, 33, 41], [22, 29, 49], [19, 20, 26, 34]] 

Done: Tue Mar 30 13:47:38 2021
